## On cherche à traduire l'ensemble du dataframe en une seule langue

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_csv('data_tokenized_2009_test_doctr', sep = '\t')

In [12]:
df.head()

,filename,type,size,width,height,mots_doctr,nb_mots,mots_concat
0,img_0000000.jpg,facture,59453,750,1061,"['factur', 'logo', 'joanner', 'binet', 'couber...",55,factur logo joanner binet coubertin pari factu...
1,img_0000001.jpg,facture,55157,750,1061,"['joanner', 'binet', 'coubertin', 'pari', 'fac...",53,joanner binet coubertin pari factur facturé ce...
2,img_0000002.jpg,facture,32097,726,1024,"['factur', 'entreprise', 'nom', 'société', 'ad...",28,factur entreprise nom société adresse postal a...
3,img_0000003.jpg,facture,63937,750,1061,"['joanner', 'binet', 'coubertin', 'pari', 'fac...",53,joanner binet coubertin pari factur cendrillon...
4,img_0000004.jpg,facture,79474,773,771,"['payer', 'ligne', 'factur', 'sfideli', 'date'...",63,payer ligne factur sfideli date création date ...


In [19]:
df.groupby('Langue').count()['filename']

Langue
UNKNOWN     16
af           6
ca          67
cy           9
de          34
en         887
es           6
et           2
fi           2
fr         410
hu          17
id          12
it          17
nl           4
no           3
pl           1
pt          33
ro          11
so          10
sq           2
sv           1
sw           1
tl          49
vi           8
Name: filename, dtype: int64

Le nombre de texte en anglais est bien plus important, on choisit donc de traduire l'intégralité en anglais.

In [20]:
from transformers import pipeline
fr_to_en = pipeline("translation_fr_to_en",model="Helsinki-NLP/opus-mt-fr-en")
i = fr_to_en("Bonjour, c'est simplement une phrase de test")

/home/clement/anaconda3/lib/python3.9/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [21]:
i

[{'translation_text': "Hello, it's just a test phrase."}]

In [22]:
i[0]['translation_text']

"Hello, it's just a test phrase."

Le test est plutôt concluant. On va le généraliser à l'ensemble du dataset. On définit une fonction de traduction, et une fonction qui détecte si le texte est un chiffre ou pas. Si c'est le cas, on le supprime.

In [23]:
def isnot_number(s):
    try:
        float(s) 
    except ValueError:
        return True
    else:
        return False
    
def traduction(liste):
    liste_traduite = []
    for i in liste:
        if isnot_number(i):
            dico = fr_to_en(i)
            liste_traduite.append(dico[0]['translation_text'])
    return liste_traduite

In [24]:
traduction(["C'est",'Une','liste','de','test','1','1.0'])

["It's", 'One', 'list', 'of the', 'test']

La colonne mots_Doctr_lemma est un string de liste. On cherche dans un premier temps à la convertir

In [25]:
import ast
df['mots_Doctr_lemma'] = df['mots_Doctr_lemma'].apply(ast.literal_eval)

In [26]:
mask = df['Langue']=='fr'
df.loc[mask,'mots_Doctr_lemma_trad'] = df.loc[mask]['mots_Doctr_lemma'].apply(traduction)

In [ ]:
mask = df['Langue']!='fr'
df.loc[mask,'mots_Doctr_lemma_trad'] = df.loc[mask]['mots_Doctr_lemma']

In [27]:
df.head()

,filename,type,size,width,height,mots_tesseract,mots_Doctr,Langue,mots_Doctr_lemma,mots_concat_tesseract,mots_concat_Doctr,mots_Doctr_lemma_trad
0,img_0000000.jpg,facture,59453,750,1061,"['facture', 'joanna', 'binet', 'coubertin', '3...","['facture', 'logo', 'joanna', 'binet', 'couber...",fr,"[factur, logo, joanner, binet, coubertin, 3140...",facture joanna binet coubertin 31400 pars fact...,factur logo joanner binet coubertin 31400 pari...,"[bill, logo, joanner, binet, coubertine, bet, ..."
1,img_0000001.jpg,facture,55157,750,1061,"['joanna', 'binet', 'coubertin', '31400', 'par...","['joanna', 'binet', 'coubertin', '31400', 'par...",fr,"[joanner, binet, coubertin, 31400, pari, factu...",joanna binet coubertin 31400 paris facturé env...,joanner binet coubertin 31400 pari factur fact...,"[joanner, binet, coubertine, bet, bill, billed..."
2,img_0000002.jpg,facture,32097,726,1024,"['facture', 'entreprise', 'nom', 'société', 'a...","['facture', 'mon', 'entreprise', 'nom', 'la', ...",fr,"[factur, entreprise, nom, société, adresse, po...",facture entreprise nom société adresse postale...,factur entreprise nom société adresse postal a...,"[bill, enterprise, name, company, Address, pos..."
3,img_0000003.jpg,facture,63937,750,1061,"['joanna', 'binet', 'coubertin', '31400', 'par...","['joanna', 'binet', 'coubertin', '31400', 'par...",fr,"[joanner, binet, coubertin, 31400, pari, factu...",joanna binet coubertin 31400 paris facture env...,joanner binet coubertin 31400 pari factur cend...,"[joanner, binet, coubertine, bet, bill, Butter..."
4,img_0000004.jpg,facture,79474,773,771,"['payer', 'ligne', 'facture', 'sfideli', 'f/05...","['payer', 'ligne', 'facture', 'sfideli', 'f/05...",fr,"[payer, ligne, factur, sfideli, f/05.2020/099,...",payer ligne facture sfideli f/05.2020/099 date...,payer ligne factur sfideli f/05.2020/099 date ...,"[pay, line, bill, sfideli, f/05/2020/099, date..."


In [34]:
print((df.iloc[0][['mots_Doctr_lemma_trad','mots_Doctr_lemma']]))

mots_Doctr_lemma_trad    [bill, logo, joanner, binet, coubertine, bet, ...
mots_Doctr_lemma         [factur, logo, joanner, binet, coubertin, 3140...
Name: 0, dtype: object


In [35]:
df.to_csv('data_tokenized_2309', sep = '\t', index = False)